# Medical Cost Project
***
***

# Goals
***
- Draw insights about what drives medical charges
- Create a machine learning model that can effectively predict medical costs

# Setup
***

In [1]:
# establishing environment
import sklearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Acquire
Acquiring the data for this project
***

In [2]:
# acquiring data from local csv
df = pd.read_csv('insurance.csv')

In [3]:
# previewing df
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
# checking for null, data types, etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


# Acquire Takeaway

- Acquired data from local csv file

# Prepare
Preparing the data for exploration and modeling

### Creating boolean columns for categorical variables

In [5]:
# creating dummy columns
df_dummies = pd.get_dummies(df, columns = ['sex', 'region' , 'smoker'], prefix = ['is','region', 'smoker'])

# renaming smoker column to match other binary column
df_dummies.rename(columns = {'smoker_yes' : 'is_smoker'}, inplace=True)

# resetting df indices otherwise merge causes extra rows to be created
df_dummies.reset_index(inplace = True)
df.reset_index(inplace = True)

# merging dummy df with main df
df = df.merge(df_dummies, how = 'inner')

# dropping these columns since their information is stored in counterpart column since they are binary values in this case
# ie. only male and female in data, if patient not female, must be male
# ie. if patient is a smoker, they must not be a non-smoker
# also dropping index here since passing drop argument to reset_index function also causes new row creation
df = df.drop(columns = ['is_male', 'smoker_no', 'index'])

df.head()

,age,sex,bmi,children,smoker,region,charges,is_female,region_northeast,region_northwest,region_southeast,region_southwest,is_smoker
0,19,female,27.900,0,yes,southwest,16884.92400,1,0,0,0,1,1
1,18,male,33.770,1,no,southeast,1725.55230,0,0,0,1,0,0
2,28,male,33.000,3,no,southeast,4449.46200,0,0,0,1,0,0
3,33,male,22.705,0,no,northwest,21984.47061,0,0,1,0,0,0
4,32,male,28.880,0,no,northwest,3866.85520,0,0,1,0,0,0


### Scale appropriate numerical columns

In [6]:
# creating scaler object
scaler = sklearn.preprocessing.MinMaxScaler()

# fitting scaler to various columns and adding scaled versions of each to DF
df['age_s'] = scaler.fit_transform(df[['age']])
df['bmi_s'] = scaler.fit_transform(df[['bmi']])
df['children_s'] = scaler.fit_transform(df[['children']])

### Moving target variable, charges, to end of df

In [7]:
# removing charges column from df and saving as variable
charges = df.pop('charges')
    
# adding back charges columns to end of DF
df['charges'] = charges

### Splitting dataframe in Train, Validate, and Test samples

In [8]:
# splitting data
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

### Previewing Train, Validate, and Test samples

In [9]:
train.head()

,age,sex,bmi,children,smoker,region,is_female,region_northeast,region_northwest,region_southeast,region_southwest,is_smoker,age_s,bmi_s,children_s,charges
1310,42,male,26.315,1,no,northwest,0,0,1,0,0,0,0.521739,0.278585,0.2,6940.90985
1264,49,female,33.345,2,no,northeast,1,1,0,0,0,0,0.673913,0.467716,0.4,10370.91255
1258,55,male,37.715,3,no,northwest,0,0,1,0,0,0,0.804348,0.585284,0.6,30063.58055
1311,33,female,26.695,0,no,northwest,1,0,1,0,0,0,0.326087,0.288808,0.0,4571.41305
1040,35,female,28.025,0,yes,northwest,1,0,1,0,0,1,0.369565,0.324590,0.0,20234.85475


In [10]:
validate.head()

,age,sex,bmi,children,smoker,region,is_female,region_northeast,region_northwest,region_southeast,region_southwest,is_smoker,age_s,bmi_s,children_s,charges
1222,50,male,25.300,0,no,southeast,0,0,0,1,0,0,0.695652,0.251278,0.0,8442.66700
287,63,female,26.220,0,no,northwest,1,0,1,0,0,0,0.978261,0.276029,0.0,14256.19280
994,27,female,20.045,3,yes,northwest,1,0,1,0,0,1,0.195652,0.109900,0.6,16420.49455
646,39,male,26.220,1,no,northwest,0,0,1,0,0,0,0.456522,0.276029,0.2,6123.56880
377,24,male,40.150,0,yes,southeast,0,0,0,1,0,1,0.130435,0.650794,0.0,38126.24650


In [11]:
test.head()

,age,sex,bmi,children,smoker,region,is_female,region_northeast,region_northwest,region_southeast,region_southwest,is_smoker,age_s,bmi_s,children_s,charges
650,49,female,42.680,2,no,southeast,1,0,0,1,0,0,0.673913,0.718859,0.4,9800.88820
319,32,male,37.335,1,no,northeast,0,1,0,0,0,0,0.304348,0.575061,0.2,4667.60765
314,27,female,31.400,0,yes,southwest,1,0,0,0,1,1,0.195652,0.415389,0.0,34838.87300
150,35,male,24.130,1,no,northwest,0,0,1,0,0,0,0.369565,0.219801,0.2,5125.21570
336,60,male,25.740,0,no,southeast,0,0,0,1,0,0,0.913043,0.263115,0.0,12142.57860


### Checking Train, Validate, Test DF dimensions

In [12]:
train.shape, validate.shape, test.shape

((749, 16), (321, 16), (268, 16))

## Exploration Takeaways

__The following changes were made to the data__

- Scale non-target variable numerical columns
    - age
    - bmi
    - children

- Create boolean columns for categorical variables
    - sex
    - smoker
    - region

- Split data in train, validate, test sets

__Notes__

- No nulls to address

- Data types are appropriate for the operations I intend to perform on them

# Explore
Exploring the data to identify drivers of medical costs and variables that can be used as features in my model